In [1]:
!pip install -q transformers==4.38.0 accelerate==0.27.0 evaluate==0.4.3 peft==0.8.2 bitsandbytes==0.42.0 torch==2.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
# the new transformer versions cause problems with quantization and fine-tune for distilbert, use 4.38.0
import transformers
print( transformers.__version__ )

4.38.0


In [3]:
import torch
import evaluate
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import(
    AutoTokenizer,
    AutoModelForSequenceClassification,
    create_optimizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)

from peft import(
    LoraConfig,
    prepare_model_for_kbit_training,
    PeftModel,
    get_peft_model,
    TaskType,
)

In [4]:
quantization_config = BitsAndBytesConfig(
        load_in_4bit = True,
        bnb_4bit_compute_dtype = torch.float16,
        bnb_4bit_quant_type = 'nf4', # Use the NormalFloat4 (NF4) quantization type, which is optimized for weights that follow a normal distribution.
        bnb_4bit_use_double_quant = True,
)

In [5]:
model_name = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

base_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels = 2,
    quantization_config = quantization_config,
)

model = prepare_model_for_kbit_training(base_model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
target_modules = set()

for name, module in base_model.named_modules():
    if isinstance(module, torch.nn.Linear):
        # Extract the names of linear layers (e.g., 'q_lin', 'k_lin', etc.)
        module_name = name.split('.')[-1]
        target_modules.add(module_name)

target_modules = list(target_modules)
print(target_modules)

['lin1', 'out_lin', 'q_lin', 'pre_classifier', 'classifier', 'lin2', 'v_lin', 'k_lin']


In [7]:
lora_config = LoraConfig(
    r = 8,
    lora_alpha = 4,
    lora_dropout = 0.05,
    bias = 'none',
    task_type = TaskType.SEQ_CLS,
    target_modules = target_modules
)

lora_model = get_peft_model(model , lora_config)

In [8]:
def load_dataset(file_path):

    df =  pd.read_csv(file_path, sep = "\t", names = ["label", "text"])

    df["label"] = df["label"].map({"ham" : 0, "spam" : 1})

    return df

In [9]:
file_path = './SMSSpamCollection.txt'
df = load_dataset(file_path)

In [10]:
df.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [11]:
def tokenize_function(text, labels):

    tokenized_text = tokenizer(text , padding = True, truncation =True, max_length = 256)
    tokenized_text['labels'] = labels

    return tokenized_text

In [19]:
from datasets import Dataset

text = df['text'].to_list()
labels = df['label'].to_list()

encodings = tokenize_function(text, labels)

encodings.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [20]:
# convert to dataset
dataset = Dataset.from_dict(encodings)

dataset = dataset.train_test_split( .2 )

In [21]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4457
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1115
    })
})

In [22]:
def metric_furction(pred_labels):

    metric = evaluate.load( 'accuracy' )
    preds, labels = pred_labels
    preds = np.argmax( preds, axis = -1 )
    acc = metric.compute( predictions = preds , references = labels )

    return acc

In [25]:
train_args = TrainingArguments(

    output_dir = './result',
    save_strategy = 'epoch',
    num_train_epochs = 1,
    learning_rate = 3e-4,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    gradient_accumulation_steps= 8,
    weight_decay = 0.01,
    evaluation_strategy = 'epoch',
    logging_dir= './logs',
    logging_strategy = 'steps',
    logging_steps = 100,
    bf16 = False,
    load_best_model_at_end = True,
    lr_scheduler_type = 'polynomial',
    warmup_ratio = 0.09,
    max_grad_norm = 1.0,
    group_by_length = True,
)


In [26]:
trainer = Trainer(
    model = lora_model ,
    args = train_args ,
    train_dataset = dataset[ 'train' ] ,
    eval_dataset = dataset[ 'test' ] ,
    compute_metrics = metric_furction
)

trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abdulrahman-ahmed20072 (abdulrahman-ahmed20072-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
0,0.145700,0.037054,0.988341


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TrainOutput(global_step=139, training_loss=0.11519925542872586, metrics={'train_runtime': 1380.736, 'train_samples_per_second': 3.228, 'train_steps_per_second': 0.101, 'total_flos': 282103029702144.0, 'train_loss': 0.11519925542872586, 'epoch': 1.0})

In [27]:
input_text = "Congratulations! You've won a $1000 Walmart gift card. Go to http://bit.ly/123456 to claim now."

inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=256)

# Perform inference
with torch.no_grad():
    outputs = lora_model(**inputs)
    logits = outputs.logits

# Get the predicted class
predicted_class = torch.argmax(logits, dim=-1).item()

# Map the predicted class to the label
label_map = {0: "ham", 1: "spam"}
predicted_label = label_map[predicted_class]

print(f"Predicted Label: {predicted_label}")

Predicted Label: spam
